<a href="https://colab.research.google.com/github/jobaydulhasanemon/python_practice/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install TensorFlow
!pip install Keras
!pip install pillow
!pip install NumPy
!pip install tqdm
!pip install jupyterlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.3/383.3 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling 

In [3]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump, load
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, Add
from keras.models import Model, load_model
from tqdm import tqdm_notebook as tqdm

tqdm().pandas()


<ipython-input-3-76bbc43d038a>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [8]:
import os
import string
from pickle import dump, load

# Load the document file into memory
def load_doc(filename):
    # Open file to read
    with open(filename, 'r') as file:
        text = file.read()
    return text

# get all images with their captions
def img_capt(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions = {}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [caption]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

# Data cleaning function will convert all upper case alphabets to lowercase, remove punctuations and words containing numbers
def txt_clean(captions):
    table = str.maketrans('', '', string.punctuation)
    for img, caps in captions.items():
        for i, img_caption in enumerate(caps):
            img_caption = img_caption.replace("-", " ")
            descp = img_caption.split()
            # uppercase to lowercase
            descp = [wrd.lower() for wrd in descp]
            # remove punctuation from each token
            descp = [wrd.translate(table) for wrd in descp]
            # remove hanging 's and a
            descp = [wrd for wrd in descp if len(wrd) > 1]
            # remove words containing numbers with them
            descp = [wrd for wrd in descp if wrd.isalpha()]
            # converting back to string
            img_caption = ' '.join(descp)
            captions[img][i] = img_caption
    return captions

# To build vocab of all unique words
def txt_vocab(descriptions):
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

# To save all descriptions in one file
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc)
    data = "\n".join(lines)
    with open(filename, "w") as file:
        file.write(data)

# Set these paths according to the project folder in your system
dataset_text = r"D:/shikha/Project - Image Caption Generator/Flickr_8k_text"
dataset_images = r"D:/shikha/Project - Image Caption Generator/Flicker8k_Dataset"

# to prepare our text data
filename = os.path.join(dataset_text, "Flickr8k.token.txt")

# Debug print statement to verify the path
print(f"Looking for the file at: {filename}")

# Check if the file exists at the specified path
if not os.path.exists(filename):
    print(f"File not found at: {filename}")
else:
    # loading the file that contains all data and map them into descriptions dictionary
    descriptions = img_capt(filename)
    print("Length of descriptions =", len(descriptions))

    # cleaning the descriptions
    clean_descriptions = txt_clean(descriptions)

    # to build vocabulary
    vocabulary = txt_vocab(clean_descriptions)
    print("Length of vocabulary =", len(vocabulary))

    # savin


Looking for the file at: D:/shikha/Project - Image Caption Generator/Flickr_8k_text/Flickr8k.token.txt
File not found at: D:/shikha/Project - Image Caption Generator/Flickr_8k_text/Flickr8k.token.txt


In [10]:
from keras.applications.xception import Xception

# Corrected line with straight quotes
model = Xception(include_top=False, pooling='avg')


83683744/83683744 [==============================] - 0s 0us/step


In [14]:
import os
from tqdm import tqdm
from keras.applications.xception import Xception
from PIL import Image
import numpy as np
from pickle import dump, load

def extract_features(directory):
    # Check if directory exists
    if not os.path.exists(directory):
        raise FileNotFoundError(f"The directory {directory} does not exist.")

    model = Xception(include_top=False, pooling='avg')
    features = {}
    for pic in tqdm(os.listdir(directory)):
        file = os.path.join(directory, pic)
        image = Image.open(file)
        image = image.resize((299, 299))
        image = np.expand_dims(image, axis=0)
        image = image / 127.5
        image = image - 1.0
        feature = model.predict(image)
        features[pic] = feature
    return features

# Path to your images directory
dataset_images = r"D:/shikha/Project - Image Caption Generator/Flicker8k_Dataset"

# Verify the path before extracting features
if not os.path.exists(dataset_images):
    print(f"The directory {dataset_images} does not exist. Please check the path.")
else:
    # Extract features
    features = extract_features(dataset_images)

    # Save features to a pickle file
    with open("features.p", "wb") as f:
        dump(features, f)

    # To directly load the features from the pickle file
    features = load(open("features.p", "rb"))


The directory D:/shikha/Project - Image Caption Generator/Flicker8k_Dataset does not exist. Please check the path.


In [18]:
import os

file_path = r"D:/shikha/Project - Image Caption Generator/Flickr_8k_text/Flickr_8k.trainImages.txt"
if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist.")


File does not exist.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')